In [1]:
## Parameters specific to where your folders are and your data
parameter_file = 'params/params.yaml'
import yaml
import sys

with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
annotation=doc['annotation']


In [2]:
tss_annotation = doc['tss_annotation']

In [3]:

import os
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm
### notebook specific configuration ###
from os.path import basename
#mpl.style.use('ggplot')
mpl.style.use('fivethirtyeight')
from cycler import cycler
mpl.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')

from Modules.Homer import *

print('Number of tissues: ',len(tissues))

('Number of tissues: ', 16)


In [4]:
genes_to_look = ['MGAT3','ID1']

In [5]:
bed_files = []
signal_files = []

In [6]:
serv = '/mnt/jabba'

In [7]:

igv_script = """new
genome {serv}/data/isshamie/genome/picr.fa
load {serv}/data/isshamie/genome/updated_final_sort.gff3

load {serv}/data/isshamie/TSS/Analysis/Results/tss_final/tss_peakCenter.bed
load {serv}/data/isshamie/TSS/Analysis/Results/tss_final/tss_peakCenter_1kb_1kb.bed

snapshotDirectory {serv}/data/isshamie/TSS/Analysis/Results/igv_snapshots

# CHO
load {serv}/data/isshamie/TSS/Processed/CHO/ATAC/out/signal/macs2/pooled_rep/CHO-ATAC-SD378_S17_R1_001.trim.nodup.tn5_pooled.pf.pval.signal.bigwig 
load {serv}/data/isshamie/TSS/Processed/CHO/GRO/bedgraph_files/trim_CHO-5GRO-SD371_S10_R1_001.bedgraph.gz 
load {serv}/data/isshamie/TSS/Processed/CHO/GRO/bedgraph_files/trim_CHO-GRO-SD369_S8_R1_001.bedgraph.gz 

# Kidney
load {serv}/data/isshamie/TSS/Processed/Kidney/GRO/bedgraph_files/trim_CHKidney_5GRO_JHS1035_SD_AGTTCC_S50_L003_R1_001.bedgraph.gz
load {serv}/data/isshamie/TSS/Processed/Kidney/GRO/bedgraph_files/trim_CHKidney_GRO_JHS1028_SD_CAACTA_S43_L003_R1_001.bedgraph.gz

# Brain
load {serv}/data/isshamie/TSS/Processed/Brain/ATAC/out/signal/macs2/rep1/CHBrain_ATAC_JHS992_SD_ACCACTGT_S4_L001_R1_001.trim.nodup.tn5.pf.pval.signal.bigwig
load {serv}/data/isshamie/TSS/Processed/Brain/GRO/bedgraph_files/trim_CHBrain_5GRO_JHS1036_SD_ATGTCA_S51_L003_R1_001.bedgraph.gz
load {serv}/data/isshamie/TSS/Processed/Brain/GRO/bedgraph_files/trim_CHBrain_GRO_JHS1029_SD_CACCGG_S44_L003_R1_001.bedgraph.gz

#Liver 
load {serv}/data/isshamie/TSS/Processed/Liver/GRO/bedgraph_files/trim_CHLiver_GRO_JHS1038_SD_GTAGAG_S53_L003_R1_001.bedgraph.gz
load {serv}/data/isshamie/TSS/Processed/Liver/ATAC/out/signal/macs2/rep1/CHLiver_ATAC_JHS994_SD_CCGTTTGT_S6_L001_R1_001.trim.nodup.tn5.pf.pval.signal.bigwig
\n
""".format(serv=serv)

In [8]:
igv_script

'new\ngenome /mnt/jabba/data/isshamie/genome/picr.fa\nload /mnt/jabba/data/isshamie/genome/updated_final_sort.gff3\n\nload /mnt/jabba/data/isshamie/TSS/Analysis/Results/tss_final/tss_peakCenter.bed\nload /mnt/jabba/data/isshamie/TSS/Analysis/Results/tss_final/tss_peakCenter_1kb_1kb.bed\n\nsnapshotDirectory /mnt/jabba/data/isshamie/TSS/Analysis/Results/igv_snapshots\n\n# CHO\nload /mnt/jabba/data/isshamie/TSS/Processed/CHO/ATAC/out/signal/macs2/pooled_rep/CHO-ATAC-SD378_S17_R1_001.trim.nodup.tn5_pooled.pf.pval.signal.bigwig \nload /mnt/jabba/data/isshamie/TSS/Processed/CHO/GRO/bedgraph_files/trim_CHO-5GRO-SD371_S10_R1_001.bedgraph.gz \nload /mnt/jabba/data/isshamie/TSS/Processed/CHO/GRO/bedgraph_files/trim_CHO-GRO-SD369_S8_R1_001.bedgraph.gz \n\n# Kidney\nload /mnt/jabba/data/isshamie/TSS/Processed/Kidney/GRO/bedgraph_files/trim_CHKidney_5GRO_JHS1035_SD_AGTTCC_S50_L003_R1_001.bedgraph.gz\nload /mnt/jabba/data/isshamie/TSS/Processed/Kidney/GRO/bedgraph_files/trim_CHKidney_GRO_JHS1028_SD_

In [9]:
vary_width = [0,100,1000,10000]

In [10]:
regions = [
['picr_2',8638555,8639172,'ID1'],
['picr_74',7465451,7465601,'MGAT3_Brain'] ,
['picr_74',7466352,7466502,'MGAT3_Kidney_1'],
['picr_74',7475954,7476104,'MGAT3_Kidney_2_Highest'],
['picr_0',5852406,5866560,'PANX2_no_CHO'],
['picr_0',5881056,5892250,'SELENOO_no_CHO'],
['picr_0',5892070,5910618,'TUBGCP6 no_CHO'],
['picr_0',6216619,6227286,'MAPK8IP2_no_CHO'],
    
['picr_0',79309482,79309669,'Merged-picr_0-79309582-3_Liver'],
['picr_0',79557138,79557303,'Merged-picr_0-79557222-7_Liver'],
['picr_0',79729634,79729788,'Merged-picr_0-79729712-2_Liver'],
['picr_1',1030823,1031038,'Merged-picr_1-1030923-11_Liver'],
['picr_1',2503367,2503587,'Merged-picr_1-2503484-11_Liver'],
['picr_1',4682533,4682713,'Merged-picr_1-4682618-11_Liver']
]

In [11]:
snaps = ''
for curr in regions:
    for w in vary_width :
        snaps = snaps + 'goto {chrom}:{start}-{end}\nsort\ncollapse\nsnapshot {name}_{w}\n'.format(
        chrom=curr[0],start=curr[1]-w,end=curr[2]+w,name=curr[3],w=w)

In [12]:
snaps

'goto picr_2:8638555-8639172\nsort\ncollapse\nsnapshot ID1_0\ngoto picr_2:8638455-8639272\nsort\ncollapse\nsnapshot ID1_100\ngoto picr_2:8637555-8640172\nsort\ncollapse\nsnapshot ID1_1000\ngoto picr_2:8628555-8649172\nsort\ncollapse\nsnapshot ID1_10000\ngoto picr_74:7465451-7465601\nsort\ncollapse\nsnapshot MGAT3_Brain_0\ngoto picr_74:7465351-7465701\nsort\ncollapse\nsnapshot MGAT3_Brain_100\ngoto picr_74:7464451-7466601\nsort\ncollapse\nsnapshot MGAT3_Brain_1000\ngoto picr_74:7455451-7475601\nsort\ncollapse\nsnapshot MGAT3_Brain_10000\ngoto picr_74:7466352-7466502\nsort\ncollapse\nsnapshot MGAT3_Kidney_1_0\ngoto picr_74:7466252-7466602\nsort\ncollapse\nsnapshot MGAT3_Kidney_1_100\ngoto picr_74:7465352-7467502\nsort\ncollapse\nsnapshot MGAT3_Kidney_1_1000\ngoto picr_74:7456352-7476502\nsort\ncollapse\nsnapshot MGAT3_Kidney_1_10000\ngoto picr_74:7475954-7476104\nsort\ncollapse\nsnapshot MGAT3_Kidney_2_Highest_0\ngoto picr_74:7475854-7476204\nsort\ncollapse\nsnapshot MGAT3_Kidney_2_Highe

In [13]:
batch = igv_script+snaps
with open('Results/igv_snapshots/igv_batch.bat','w') as f:
    f.write(batch)

----------

----------

In [ ]:
bed_regions_of_interest = ['MGAT3']

In [20]:
'goto {chrom}:{start}-{end}\nsort\ncollapse\nsnapshot {name}'.format(
    chrom=curr[0],start=curr[1],end=curr[2],name=curr[3])

'goto picr_2:8638555-8639172\nsort\ncollapse\nsnapshot ID1'